# Pre-requisites and Base Model Load


In [ ]:
#Install the required packages for this project
!pip install einops datasets bitsandbytes accelerate peft flash_attn
!pip uninstall -y transformers
!pip install transformers
!pip install sklearn
!pip install wandb
!pip install logging
!pip install --upgrade torch
!nvidia-smi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 79.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-m

### Load Base Model & Adapter



In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
from huggingface_hub import notebook_login

os.environ['HF_HOME'] = 'REDACTED'

model_name = "microsoft/Phi-3-medium-4k-instruct"
# Configuration to load model in 4-bit quantized
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_compute_dtype='float16',
                                #bnb_4bit_compute_dtype=torch.bfloat16,
                                bnb_4bit_use_double_quant=True)


#Loading Microsoft's Phi-3 model with compatible settings
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto',
                                             quantization_config=bnb_config,
                                             attn_implementation="flash_attention_2",
                                             trust_remote_code=True)

# Setting up the tokenizer for Phi-3
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          add_eos_token=True,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"

print(f"Memory footprint: {model.get_memory_footprint() / 1e9} GB")

print(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/934 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-medium-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Memory footprint: 7.47324416 GB
Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 5120, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-39): 40 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (qkv_proj): Linear4bit(in_features=5120, out_features=7680, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=5120, out_features=35840, bias=False)
          (down_proj): Linear4bit(in_features=17920, out_features=5120, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_

## Tokenize and Prep Dataset for Training

### Phi-2 Dataset from Google Drive
JSON library based on three dictionary entries: "prompt" "chosen" and "references"

In [ ]:
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import json
from google.colab import drive
import os

# This line is necessary to mount Google Drive in Colab
drive.mount('/content/drive')

# Define the base directory in Google Drive where your project files are stored
base_directory = '/content/drive/My Drive/Colab Notebooks'
data_directory = os.path.join(base_directory, "finetuning/data/medical_gpt2_struct/")

def load_jsonl(path):
    with open(path, 'r') as file:
        return [json.loads(line) for line in file]

def train_test_split_data(data, test_size=0.2):
    return train_test_split(data, test_size=test_size, random_state=42)
    # note random state = 42 on all assessment data
    #### NOTE RANDOM STATE 42

# Load data and split into training and testing sets
data_path = os.path.join(data_directory, "combined_textbooks_Training.jsonl")
data = load_jsonl(data_path)
train_data, test_data = train_test_split_data(data)

# Convert to Hugging Face datasets
train_dataset = Dataset.from_dict({"text": train_data}) # was "data" - this and all flags below dated 3Mar24 15:27
test_dataset = Dataset.from_dict({"text": test_data}) # was "data"

# Note tokenize function was defined and called here previously (replaced below 24 Feb)

def collate_and_tokenize(examples):
    processed_examples = {'input_ids': [], 'attention_mask': [], 'labels': []}
    for example in examples['text']: # was data
        # Split the 'text' content into the required segments
        text = example['text']  # Directly use the 'text' field from your dataset
        # Tokenize the text
        encoded = tokenizer(
            text,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=1024,  # Originally 2048 (can be reduced to save memory)
        )
        # Append the tokenized information to the processed examples
        processed_examples['input_ids'].append(encoded['input_ids'][0])
        processed_examples['attention_mask'].append(encoded['attention_mask'][0])
        processed_examples['labels'].append(encoded['input_ids'][0])  # Assuming labels are the same as inputs

    # Convert lists to tensors
    processed_examples['input_ids'] = torch.stack(processed_examples['input_ids'])
    processed_examples['attention_mask'] = torch.stack(processed_examples['attention_mask'])
    processed_examples['labels'] = torch.stack(processed_examples['labels'])

    return processed_examples

print(test_dataset['text'][0]) # was 'data'

#We will just keep the input_ids and labels that we add in function above.
columns_to_remove = ["text"] # was data

#tokenize the training and test datasets
tokenized_dataset_train = train_dataset.map(collate_and_tokenize,
                                            batched=True,
                                            batch_size=1,
                                            remove_columns=columns_to_remove)
tokenized_dataset_test = test_dataset.map(collate_and_tokenize,
                                          batched=True,
                                          batch_size=1,
                                          remove_columns=columns_to_remove)

#Check if tokenization looks good
input_ids = tokenized_dataset_train[0]['input_ids'][0]  # Adjust indexing based on actual structure

decoded = tokenizer.decode(input_ids, skip_special_tokens=True)
print(decoded)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'text': '### Query: How is 1,25-dihydroxycholecalciferol synthesized and what are its primary functions in the body? ### Response: 1,25-dihydroxycholecalciferol is primarily synthesized in the kidneys from 25-hydroxycholecalciferol and plays a crucial role in regulating calcium and phosphate homeostasis. It stimulates the expression of gene products involved in calcium transport and absorption, facilitating calcium reabsorption in the kidneys and increasing calcium absorption from the intestine. ### References: 1,25-dihydroxycholecalciferol is synthesized primarily in the kidneys from 25-hydroxycholecalciferol. It regulates calcium and phosphate homeostasis by stimulating gene products involved in calcium transport. It facilitates calcium reabsorption in the kidneys and increases calcium absorption from the intestine. ###'}


Map:   0%|          | 0/27847 [00:00<?, ? examples/s]

Map:   0%|          | 0/6962 [00:00<?, ? examples/s]

### Phi-3 Dataset from Google Drive
JSON library based on three dictionary entries: "prompt" "chosen" and "references"

In [ ]:
import os
import json
from datasets import Dataset
from sklearn.model_selection import train_test_split
from google.colab import drive
import hashlib
from transformers import AutoTokenizer

# Mount Google Drive
drive.mount('/content/drive')

# Define the base directory in Google Drive where your project files are stored
base_directory = '/content/drive/My Drive/Colab Notebooks'
#data_directory = os.path.join(base_directory, "finetuning/data/medical_gpt2_struct/") # for medical training sources
data_directory = os.path.join(base_directory, "finetuning/data/unitops_gpt2_struct/") # for assessment training sources

# Load JSONL data
def load_jsonl(path):
    with open(path, 'r') as file:
        return [json.loads(line) for line in file]

def train_test_split_data(data, test_size=0.2):
    return train_test_split(data, test_size=test_size, random_state=42)

# Load and split data
#data_path = os.path.join(data_directory, "combined_textbooks_Training.jsonl") # for medical training sources
data_path = os.path.join(data_directory, "combined_UnitOps_Training_ALL.jsonl") # for assessment training sources
data = load_jsonl(data_path)
train_data, test_data = train_test_split_data(data)

# Convert to Hugging Face datasets
train_dataset = Dataset.from_dict({"text": [item['text'] for item in train_data]})
test_dataset = Dataset.from_dict({"text": [item['text'] for item in test_data]})

# Initialize the tokenizer for the Phi-3 model (commented out with above)
#model_name = "microsoft/Phi-3-mini-4k-instruct"
#tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          #add_eos_token=True,
                                          #trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"

# Function to format data in the UltraChat 200k structure
def format_ultrachat_data(data):
    formatted_data = []
    for item in data:
        text = item['text']
        # Extracting parts from the text
        query_start = text.find("### Query:") + len("### Query:")
        response_start = text.find("### Response:") + len("### Response:")
        references_start = text.find("### References:") + len("### References:")

        query = text[query_start:response_start - len("### Response:")].strip()
        response = text[response_start:references_start - len("### References:")].strip()

        # Creating a unique prompt_id
        prompt_id = hashlib.sha256(query.encode()).hexdigest()

        # Constructing the new format
        formatted_item = {
            "prompt": query,
            "prompt_id": prompt_id,
            "messages": [
                {"content": query, "role": "user"},
                {"content": response, "role": "assistant"}
            ]
        }
        formatted_data.append(formatted_item)
    return formatted_data

# Format the data
train_data_formatted = format_ultrachat_data(train_data)
test_data_formatted = format_ultrachat_data(test_data)

# Convert to Hugging Face datasets
train_dataset = Dataset.from_dict({"data": train_data_formatted})
test_dataset = Dataset.from_dict({"data": test_data_formatted})

# Tokenization function
def collate_and_tokenize(examples):
    processed_examples = {'input_ids': [], 'attention_mask': [], 'labels': []}
    for example in examples['data']:
        messages = example['messages']
        text = " ".join([msg['content'] for msg in messages])

        # Tokenize the text
        encoded = tokenizer(
            text,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=4096,
        )
        # Append the tokenized information
        processed_examples['input_ids'].append(encoded['input_ids'][0])
        processed_examples['attention_mask'].append(encoded['attention_mask'][0])
        processed_examples['labels'].append(encoded['input_ids'][0])

    # Convert lists to tensors
    processed_examples['input_ids'] = torch.stack(processed_examples['input_ids'])
    processed_examples['attention_mask'] = torch.stack(processed_examples['attention_mask'])
    processed_examples['labels'] = torch.stack(processed_examples['labels'])

    return processed_examples

# Tokenize the datasets
tokenized_dataset_train = train_dataset.map(collate_and_tokenize, batched=True)
tokenized_dataset_test = test_dataset.map(collate_and_tokenize, batched=True)

# Verify the tokenization
input_ids = tokenized_dataset_train[0]['input_ids']
decoded = tokenizer.decode(input_ids, skip_special_tokens=True)
print(decoded)

Mounted at /content/drive


Map:   0%|          | 0/3595 [00:00<?, ? examples/s]

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

What is the significance of the Reynolds number in determining flow regimes within pipes? The Reynolds number is a dimensionless quantity that plays a crucial role in characterizing flow regimes within pipes. It represents the ratio of inertial forces to viscous forces in a fluid system. When the Reynolds number is below 2100, laminar flow dominates, characterized by smooth, orderly fluid motion. Above 4000, turbulent flow prevails, marked by chaotic fluid movements. Between these values, the flow is in a transition state, exhibiting characteristics of both laminar and turbulent flow. Understanding the Reynolds number helps engineers design efficient piping systems by predicting the flow behavior.


In [ ]:
# OPTIONAL - Random verification of training dataset input
input_ids = tokenized_dataset_train[256]['input_ids']
decoded = tokenizer.decode(input_ids, skip_special_tokens=True)
print(decoded)

In the context of Unit Operations, how does the concept of void fraction apply to packed beds? Void fraction is a dimensionless quantity representing the fraction of the volume of a packed bed that is not occupied by the packing material. It is crucial for understanding how fluid flows through the bed and for the design and analysis of operations such as gas absorption, distillation, or liquid-liquid extraction in packed columns. A higher void fraction indicates more space for fluid flow, which can affect the pressure drop across the bed and the efficiency of mass transfer between the fluid phases. Engineers use the void fraction to optimize the design of packed beds to ensure effective contact between phases for optimal mass transfer.


# Model Training

In [ ]:
for name, module in model.named_modules():
    print(name, module)

 Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3SuScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_feature

### Training Design for Phi-2

In [ ]:
#Accelerate training models on larger batch sizes, we can use a fully sharded data parallel model.
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )


print_trainable_parameters(model)

#gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# Freeze base model layers and cast layernorm in fp32
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
print(model)

### LORA config for Phi-2; r at 512 = 33.17%
config = LoraConfig(
    r=512,
    lora_alpha=32,
    target_modules=[
    'q_proj',
    'k_proj',
    'v_proj',
    'dense',
    'fc1',
    'fc2',
    ], #print(model) will show the modules to use
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

lora_model = accelerator.prepare_model(lora_model)

### Training Design for Phi-3

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

# Define FSDP plugin configuration
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

# Initialize the accelerator with FSDP plugin
accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

# Print initial model parameters
print_trainable_parameters(model)

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
print(model)

# Define the target modules for LoRA based on Phi-3's architecture
target_modules = []
for i in range(32):  # Confirmed - 32 layers in Phi-3
    target_modules.extend([
        f'model.layers.{i}.self_attn.o_proj',
        f'model.layers.{i}.self_attn.qkv_proj',
        f'model.layers.{i}.mlp.gate_up_proj',
        f'model.layers.{i}.mlp.down_proj',
    ])

# LoRA Configuration (r=256: 16.7% phi3 trainable & r=128: 9.82% trainable)
config = LoraConfig(
    r=64, #was 64 w/flash attn2 (128 prev), reduced for memory footprint on Phi-3-medium
    lora_alpha=32, # was 32
    target_modules=target_modules,
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

# Apply PEFT LoRA configuration to the model
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

# Prepare model with the accelerator
lora_model = accelerator.prepare_model(lora_model)

trainable params: 328750080 || all params: 7144494080 || trainable%: 4.60
Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 5120, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-39): 40 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (qkv_proj): Linear4bit(in_features=5120, out_features=7680, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=5120, out_features=35840, bias=False)
          (down_proj): Linear4bit(in_features=17920, out_features=5120, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )


### Training Arguments

In [ ]:
import time
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import wandb
import logging

# Uncomment to initialise WANDB tracking
wandb.init(project="phi3-unitops", entity="zachaman")
logging.basicConfig(level=logging.INFO)
logging.getLogger("codecarbon").setLevel(logging.INFO)

# batch size 8 @ tokenizer <= 512 = 17 GB RAM
# batch size 16 @ tokenizer <= 512 = 30.5 GB GPU RAM
# batch size 4 @ tokenizer <= 2048 = 34 GB GPU RAM
# note flash2 + 4096 context + 4 batch = 39.5 GB GPU RAM
# 5 epochs @ assessment data size (3700 QAR triplets @ 100 words) @ 2.5 e-6 works well (1 vs 4 e-6 hard bounds)
training_args = TrainingArguments(
    output_dir='./outputs',  # Output directory for checkpoints and predictions
    overwrite_output_dir=True, # Overwrite the content of the output directory
    per_device_train_batch_size=1,  # Batch size for training
    per_device_eval_batch_size=1,  # Batch size for evaluation
    gradient_accumulation_steps=4, # number of steps before optimizing
    gradient_checkpointing=True,   # Enable gradient checkpointing
    gradient_checkpointing_kwargs={"use_reentrant": False},
    warmup_steps=100,  # Number of warmup steps
    #max_steps=100,  # Total number of training steps
    num_train_epochs=5,  # Number of training epochs
    learning_rate=1e-6,  # Learning rate
    weight_decay=0.01,  # Weight decay
    optim="paged_adamw_8bit", #Keep the optimizer state and quantize it
    fp16=True, #Use mixed precision training
    #For logging and saving
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=5,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=10,  # Limit the total number of checkpoints
    eval_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True, # Load the best model at the end of training
    #run_name="user-specified-X" # Uncomment to override WANDB auto-assignment
)

# Define early stopping callback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3  # Number of evaluation steps to wait for improvement
)

trainer = Trainer(
    model=lora_model,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    args=training_args,
    callbacks=[early_stopping_callback],  # Include early stopping callback
)

#Disable cache to prevent warning, renable for inference
model.config.use_cache = False

start_time = time.time()  # Record the start time
trainer.train()  # Start training
end_time = time.time()  # Record the end time

training_time = end_time - start_time  # Calculate total training time

print(f"Training completed in {training_time} seconds.")

wandb.finish() # NOTE - SYNC THIS COMMENT
# WANDB Key: 8ce3e407e377168b3f3384b88fed78add0f40ccc

# Note that training was not turned off at the end of this

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.50 GiB. GPU 

## Adapter Export

### Save to Google Drive

In [ ]:
# Need to introduce local (Drive+)
from google.colab import drive
import os
drive.mount('/content/drive')
save_path = '/content/drive/My Drive/Colab Notebooks/finetuning/phi3_splendidglitter11'

# confirm existence of savepath
if not os.path.exists(save_path):
    os.makedirs(save_path)

lora_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/My Drive/Colab Notebooks/finetuning/phi3_splendidglitter11/tokenizer_config.json',
 '/content/drive/My Drive/Colab Notebooks/finetuning/phi3_splendidglitter11/special_tokens_map.json',
 '/content/drive/My Drive/Colab Notebooks/finetuning/phi3_splendidglitter11/tokenizer.model',
 '/content/drive/My Drive/Colab Notebooks/finetuning/phi3_splendidglitter11/added_tokens.json',
 '/content/drive/My Drive/Colab Notebooks/finetuning/phi3_splendidglitter11/tokenizer.json')

### Push to HuggingFace Hub

In [ ]:
from huggingface_hub import notebook_login

#Save model to hub to ensure we save our work.
hf_write = 'REDACTED'
notebook_login()

In [ ]:
private_repo = "zachaman/phi3-splendidglitter11"

lora_model.push_to_hub(private_repo,
                  use_auth_token=True,
                  commit_message="phi3 qlora model commit",
                  private=True)
# have not debugged tokenizer push
tokenizer.push_to_hub(private_repo,
                      use_auth_token=True,
                      commit_message="Tokenizer for phi3 qlora",
                      private=True)
# TODO: manually transfer lora_adapter and config file (this method merges and exports a single .safetensors model file)


/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/713M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/zachaman/phi3-splendidglitter11/commit/129cc0f86232e6da5133358d6d294bed8591e4d1', commit_message='Tokenizer for phi3 qlora', commit_description='', oid='129cc0f86232e6da5133358d6d294bed8591e4d1', pr_url=None, pr_revision=None, pr_num=None)

## Merge and Export

In [ ]:
# Merge the LoRA adapter with the base model
merged_model = lora_model.merge_and_unload()

### Export Merge to Google Drive

In [ ]:
# Save the merged model
merged_model_save_path = "/content/drive/My Drive/Colab Notebooks/finetuning/phi3_splendidglitter11/merged"
merged_model.save_pretrained(merged_model_save_path)
tokenizer.save_pretrained(merged_model_save_path)

### Export to HuggingFace

In [ ]:
from huggingface_hub import notebook_login

#Save model to hub to ensure we save our work.
hf_write = 'REDACTED'
notebook_login()

In [ ]:
private_repo = "zachaman/chpr3019"

merged_model.push_to_hub(private_repo,
                  use_auth_token=True,
                  commit_message="phi3 4k medium qlora merged; trained on UnitOps ZAR combined dataset",
                  private=False)
# have not debugged tokenizer push
tokenizer.push_to_hub(private_repo,
                      use_auth_token=True,
                      commit_message="Tokenizer for phi3 4k medium qlora merged; trained on UnitOps ZAR combined dataset",
                      private=False)
# TODO: manually transfer lora_adapter and config file (this method merges and exports a single .safetensors model file)


### ONNX Conversion

Use Transformers <b>Optimum-CLI</b> for direct converstion across Google Drive (confirmed working):

In [ ]:
!optimum-cli export onnx --model /content/drive/My\ Drive/Colab\ Notebooks/finetuning/phi3_splendidglitter11/merged/ --task text-generation --opset 14 /content/drive/My\ Drive/Colab\ Notebooks/finetuning/phi3_splendidglitter11/merged_onnx/onnx_model.onnx

In [ ]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForCausalLM
import torch

# Path to the exported ONNX model
onnx_model_path = "/content/drive/My Drive/Colab Notebooks/finetuning/phi3_stellargalaxy8/merged_onnx/onnx_model.onnx"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(onnx_model_path)

# Load the ONNX model with use_cache=False and use_io_binding=False
model = ORTModelForCausalLM.from_pretrained(onnx_model_path, use_cache=False, use_io_binding=False)

# Example input for testing the model
input_text = "Write a question and answer for a postgraduate emergency medicine exam."
inputs = tokenizer(input_text, return_tensors="pt")  # Inputs are on CPU

# Generate output from the ONNX model
outputs = model.generate(**inputs, max_new_tokens=500)  # Specify max_new_tokens to control generation length
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(result)


## Stop Runtime

In [ ]:
#Terminate the session
from google.colab import runtime
runtime.unassign()